In [ ]:
#This script takes every simulation with all points and calculates rmse and mae in each case

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('../machine-scientist/')
sys.path.append('../machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [5]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param, dimensions=False):

    if dimensions==True:
         VARS = ['x','y',]
         prior_par = read_prior_par('../machine-scientist/Prior/final_prior_param_sq.named_equations.nv2.np10.2016-09-09 18:49:42.600380.dat')
    else:
        VARS = ['x',]
        prior_par = read_prior_par('../machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')

    try:
        x = dataframe[[c for c in VARS]].copy()
        y = dataframe.z_noise
    except KeyError:
        VARS = ['x1',]
        x = dataframe[[c for c in VARS]].copy()
        y = dataframe.y_noise

    

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    
    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dn)
    dplot['zbms'] = t.predict(x)

    return dplot
    

In [6]:
#Read NN and BMS data
functions=['1', '5' , '7', '8', '10']

realizations=2
N=9
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
resolution='1x' #0.5x, 1x, 2x

if resolution=='1x':
    train_size=60
elif resolution=='0.5x':
    train_size=30
elif resolution=='2x':
    train_size=120

runid=0
NPAR=10 #10, 20
steps=50000

rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[];function_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for function in functions:

    for sigma in sigmas:

        for r in range(realizations+1):
            
            file_model='NN_no_overfit_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            model_d='../../data/nguyen/' + resolution + '_resolution/trained_nns/' + file_model
            d=pd.read_csv(model_d)

            n_index.append(function);r_index.append(r);sigma_index.append(sigma);function_index.append(function)


            dn=d[d['rep']==int(function)]
            dn=clean_index(dn)

            #Read BMS data
            if resolution=='1x':
                filename='BMS_nguyen_n_' + function +'_sigma_'+str(sigma)+ '_r_' + str(r) + '_id_0_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
            elif resolution=='0.5x':
                filename='BMS_nguyen_n_' + str(n) +'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.1_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
            elif resolution=='2x':
                filename='BMS_nguyen_n_' + str(n) +'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.025_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                    
            trace=pd.read_csv('../../data/MSTraces/nguyen/' + resolution + '_resolution/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])

            #distinguish between one variable and two-variable functions
            if function=='10':
                dplot=add_bms_pred(dn, trace, NPAR, dimensions=True)
            else:
                dplot=add_bms_pred(dn, trace, NPAR)
    
            #Errors
            
            #nns
            rmse_nn_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['zmodel'],dplot.loc[:train_size -1]['z'])
            rmse_nn_train.append(rmse_nn_train_i)
            
            rmse_nn_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['zmodel'],dplot.loc[train_size -1:]['z'])
            rmse_nn_test.append(rmse_nn_test_i)

            mae_nn_train_i=mean_absolute_error(dplot.loc[:train_size-1]['zmodel'],dplot.loc[:train_size -1]['z'])
            mae_nn_train.append(mae_nn_train_i)
            
            mae_nn_test_i=mean_absolute_error(dplot.loc[train_size-1:]['zmodel'],dplot.loc[train_size -1:]['z'])
            mae_nn_test.append(mae_nn_test_i)

            #bms
            try:
                rmse_mdl_i=mean_squared_error(dplot.zbms,dn.z)
            except ValueError:
                rmse_mdl_i=np.inf
            
            rmse_mdl_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['zbms'],dn.loc[:train_size-1]['z'])
            rmse_mdl_train.append(rmse_mdl_train_i)

            try:
                rmse_mdl_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['zbms'],dn.loc[train_size-1:]['z'])
            except ValueError:
                rmse_mdl_test_i=np.inf
                
            rmse_mdl_test.append(rmse_mdl_test_i)

            mae_mdl_train_i=mean_absolute_error(dplot.loc[:train_size-1]['zbms'],dplot.loc[:train_size -1]['z'])
            mae_mdl_train.append(mae_mdl_train_i)

            try:
                mae_mdl_test_i=mean_absolute_error(dplot.loc[train_size-1:]['zbms'],dplot.loc[train_size -1:]['z'])
            except ValueError:
                mae_mdl_test_i=np.inf
                
            mae_mdl_test.append(mae_mdl_test_i)

errors_df=pd.DataFrame({'sigma':sigma_index, 'function':function_index, 'mae_nn_train':mae_nn_train, 'mae_nn_test':mae_nn_test, 'mae_mdl_train':mae_mdl_train, 
                        'mae_mdl_test':mae_mdl_test, 'rmse_nn_train':rmse_nn_train, 'rmse_nn_test': rmse_nn_test, 
                        'rmse_mdl_train':rmse_mdl_train, 'rmse_mdl_test': rmse_mdl_test, 'n':n_index, 'r': r_index})
errors_df.to_csv('../../data/all_errors_nguyen' +  '.csv')
display(errors_df)

sigma function  mae_nn_train  mae_nn_test  mae_mdl_train  mae_mdl_test  \
0     0.00        1      0.041645     1.938429   1.355397e-16  7.190016e-16   
1     0.00        1      0.049886     2.105638   1.405889e-16  6.344132e-16   
2     0.00        1      0.051987     2.129139   1.231654e-16  9.093255e-16   
3     0.02        1      0.038515     1.926238   3.238400e-03  2.129294e-01   
4     0.02        1      0.058483     2.192715   1.749198e-03  1.656911e-02   
..     ...      ...           ...          ...            ...           ...   
160   0.18       10      0.024106     0.078632   3.500288e-02  4.373102e-02   
161   0.18       10      0.020973     0.101032   3.348199e-02  4.278379e-02   
162   0.20       10      0.059975     0.254647   3.728207e-02  1.149081e-01   
163   0.20       10      0.028298     0.034410   8.814254e-02  6.073678e-02   
164   0.20       10      0.037908     0.062005   3.711375e-02  1.302092e-01   

     rmse_nn_train  rmse_nn_test  rmse_mdl_train  rmse_mdl_test   n  r  
0         0.096256      2.273878    2.103031e-16   1.130134e-15   1  0  
1         0.116285      2.438433    2.134518e-16   1.016382e-15   1  1  
2         0.121103      2.458621    2.059831e-16   1.128055e-15   1  2  
3         0.091641      2.270680    6.415863e-03   2.715548e-01   1  0  
4         0.133382      2.522208    2.392921e-03   1.797497e-02   1  1  
..             ...           ...             ...            ...  .. ..  
160       0.032599      0.087837    4.195916e-02   5.652054e-02  10  1  
161       0.028642      0.110934    3.753406e-02   5.535585e-02  10  2  
162       0.071429      0.277439    4.690686e-02   1.310190e-01  10  0  
163       0.034189      0.042639    1.036397e-01   6.699369e-02  10  1  
164       0.049311      0.072216    5.135096e-02   1.446254e-01  10  2  

[165 rows x 12 columns]